<a href="https://colab.research.google.com/github/SHIELDXIE/stylegan2-train-colab/blob/main/Lecture4_StyleGAN2_ADA-update1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA-PyTorch


## 0.1 环境搭建

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


查看分配到的GPU，如果分配到P100基本就够用了，V100更好。

In [ ]:
!nvidia-smi -L


GPU 0: Tesla T4 (UUID: GPU-b8077889-aa54-249a-f71d-f7db84a76450)


##1.1 安装代码库

下一个单元格将会安装styleGAN2-ada的代码库（一个文件夹）到你的GoogleDrive里。网页版GoogleDrive里会在AI文件夹下看到新建的stylegan2-ada-pytorch文件夹。

如果你已经安装过了，当前运行路径会移动到文件夹里。

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/AI/stylegan2-ada-pytorch"):
    %cd "/content/drive/MyDrive/AI/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir AI
    %cd AI
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir result
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir result
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

/content/drive/MyDrive/AI/stylegan2-ada-pytorch
     |████████████████████████████████| 108 kB 26.0 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 12.8 MB 68.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.


下面单元格是非必须运行的。其作用是更新代码库到原作者最新的版本。

In [ ]:
# %cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git pull
# !git stash
# !git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

个人习惯，重新检查一遍当前路径是否正确。

此时你应该看到 ....../stylegan2-ada-pytorch

In [ ]:
!pwd

/content


##1.1 数据集准备

图片的批量处理建议使用软件XnViewMP。

上传由方形图片(512px或者1024px)组成的数据集，并且压缩成一个**.zip**文件。

在电脑压缩完成之后上传到网页版本的GoogleDrive，具体路径是**AI/stylegan2-ada-pytorch/datasets**

##1.2 训练模型

以下是你在训练模型前需要设置的参数。但是许多都已经设置好了默认值，无需更改。

比较重要的几个参数如下，
* `dataset_path`: 数据集.zip的具体路径。可以通过侧边栏右键获得。
* `resume_from`: 如果你是训练一个全新的数据集，作者建议使用 `'ffhq1024'` 或者 `'./pretrained/wikiart.pkl'` 进行迁移学习。（需要保证你的数据集和模型中训练的数据集图片大小一致）
* `mirror_x` 和 `mirror_y`: 是否在水平方向和竖直方向作数据集图片的对称。

In [ ]:
#required: definitely edit these!
dataset_path = 'datasets/dataset.zip'
resume_from =   '/content/drive/MyDrive/AI/stylegan2-ada-pytorch/result/abstract_animal/00007-dataset-mirror-paper512-gamma50-bgc-resumeffhq512/network-snapshot-000000.pkl' #'../../pretrained_model/liquid_art-512.pkl' 
outdir = 'result/abstract_animal'    
aug_strength = 2
train_count = 0
mirror_x = True
mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bgc'#或者bgc
# 'blit', 'geom', 'color', 'filter', 'noise', 'cutout', 'bg', 'bgc', 'bgcf', 'bgcfn', 'bgcfnc'
config = 'paper512'
#'auto', '11gb-gpu', '11gb-gpu-complex', '24gb-gpu', '24gb-gpu-complex', '48gb-gpu', '48gb-2gpu', 'stylegan2', 'paper256', 'paper512', 'paper1024', 'cifar', 'cifarbaseline', 'aydao'
snapshot_count = 2

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=$outdir --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=$mirror_y --nkimg=$train_count


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 2,
  "network_snapshot_ticks": 2,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "datasets/dataset.zip",
    "use_labels": false,
    "max_size": 2490,
    "xflip": true,
    "resolution": 512
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 8
    },
    "channel_base": 32768,
    "channel_max": 512,
    "num_fp16_res": 4,
  

### 恢复训练

如果你的Colab训练到一半断开了，需要连上Colab后继续你的训练。主要需要修改两个参数，分别是 `resume_from` 和 `aug_strength` 。

1. 将 `resume_from` 参数设置为最后的**.pkl**文件 (在 `results` 文件夹会看到，都是你训练出来的模型文件)。
2. 更新 `aug_strength` 值。打开训练日志**log.txt**，查看前面.pkl文件对应的augment值，然后填上。

设置完成后，继续执行训练。

###Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/runway.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/runway.pkl

##2.0 测试/推理 Testing/Inference

测试、使用你的模型。一般用来生成图片和视频。

### 2.1 生成单张图片

`--network`: 你使用的.pkl文件路径。

`--seeds`: 选择你的随机种子。不同的随机种子会得出不同的结果。

`--truncation`: 截断, 用来截断你的潜在空间latent space。 一般来说，值越小，生成的照片就越“正常真实”（这里理解为结果接近你的数据集），但是多样性也会减少。一般使用0.5-1中间的值，实际上是无限制的数。

In [ ]:
network = '../../result/ukiyoe/00000-ukiyoe-1024-v2-pytorch-mirror-paper1024-gamma50-bg-resumecustom/network-snapshot-000016.pkl'
outdir = '../../result/ukiyoe/generate-t1.5-016pkl'
!python generate.py --outdir=$outdir --trunc=1.5 --seeds=0-200 --network=$network

We can use these options for any image or video generation commands (excluding projection).

### 2.2 遍历截断 Truncation Traversal

下面你可以选择一个随机种子，然后看在该随机种子下，不同截断值之间会有什么变化。-1到1之间会有比较真实的图片，偏离越多图片就会越奇怪。
#### 参数选项 Options 
`--network`: 你的.pkl文件。

`--seeds`: 选择一个随机种子。

`--start`: 初始截断值。

`--stop`: 终止截断值，需要大于初始截断值，否则可能发生错误。

`--increment`: 每一帧之间的变化大小。如果想要视频长一点，就设置小一点，反之亦然。(stop-start/increment=total frames)


In [ ]:
!python generate.py --process="truncation" --outdir=/content/out/trunc-trav-3/ --start=-0.8 --stop=0.8 --increment=0.02 --seeds=470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

### 2.3 插值 Interpolations

插值是给一个向量产生非常小的变化的过程，可以用来做从一帧到另一帧的变化。
Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options

`--network`: 你的.pkl文件。

`--interpolation`: 插值的类型。在z space和w space的区别。一般看不出区别。

`--frames`: 帧数，控制视频长短。

`--trunc`: 截断值。

#### 2.3.1 线性插值 Linear Interpolation

In [ ]:
!python generate.py --outdir=/content/out/video1-w-0.5/ --space="z" --trunc=0.5 --process="interpolation" --seeds=463,470 --network=/content/drive/MyDrive/stylegan2-transfer-models/mixed6k-network-snapshot-016470.pkl

In [ ]:
!python generate.py --outdir=out/video1-w/ --space="w" --trunc=1 --process="interpolation" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

####2.3.2 Slerp插值

另外一种插值方式。从技术原理分析，线性插值在高维的GAN空间中并不是最好的。[This github link](https://github.com/soumith/dcgan.torch/issues/14) 有一些比较主流的观点和讨论。

实际中找不出很大的区别。 (the difference in z- and w-space is enough in many cases)

In [ ]:
!python generate.py --outdir=out/slerp-z/ --space="z" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=24

In [ ]:
!python generate.py --outdir=out/slerp-w/ --space="w" --trunc=1 --process="interpolation" --interpolation="slerp" --seeds=85,265,297,849 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl --frames=12

####2.3.3 噪声循环 Noise Loop

在z space产生一个随机的插值视频（目前只适用于z space）。

`--interpolation="noiseloop"`: 设置好插值方式为noise loop。

`--diameter`: “直径”。值越小，视频中展现出的图片多样性就越小；值越大，视频中展现出的图片多样性就越大。结合 `--frames` 可以帮助你调节视频的节奏。

`--random_seed`: 决定你在z space的初始位置。

In [ ]:
!python generate.py --outdir=out/video-noiseloop-0.9d/ --trunc=0.8 --process="interpolation" --interpolation="noiseloop" --diameter=0.9 --random_seed=100 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

####2.3.4 回路循环 Circular Loop

Noise Loop会比较杂乱，Circular相对平稳一点。

作者建议用比较大的 `--diameter` （相对于Noise Loop所使用的）。比如50到500之间。

`--frames` 可以控制速度和多样性。

In [ ]:
!python generate.py --outdir=out/video-circularloop/ --trunc=1 --process="interpolation" --interpolation="circularloop" --diameter=800.00 --frames=720 --random_seed=90 --network=/content/stylegan2-ada-pytorch/pretrained/wikiart.pkl

##3.0 投射 Projection

###3.1 基础投射 Basic Projector

*   `--target`: 你的目标图片。模型会在自己的空间中“找到”这张图片（尽可能一致）。目标图片的大小必须和模型的图片大小一致。
*   `--num-steps`: 步数。步数越大，找到相似度高的可能性就越高。



In [ ]:
!python projector.py --help

In [ ]:
!python projector.py --network=/content/drive/MyDrive/pretrained_model/abstract_art.pkl --outdir=/content/projector/ --target=/content/001-0.png --num-steps=200 --seed=0

###3.2 Peter Baylies’ Projector

In [ ]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --help

In [ ]:
!python /content/stylegan2-ada-pytorch/pbaylies_projector.py --network=/content/drive/MyDrive/pretrained_model/abstract_art.pkl --outdir=/content/projector-no-clip-006265-4-inv-3k/ --target-image=/content/001-0.png --num-steps=3000 --use-clip=False --use-center=False --seed=99